# Data cleaning 

In [1]:
import pandas as pd
import os

import numpy as np

In [2]:
from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
# 
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals
from use_cases.pairs import create_pair_token
from use_cases.persons_dialogues import create_table_persons_dialogues

import use_cases.utils.textools as tt


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading Data Surveys

In [3]:
# chunksize   = 100 # Just for debugging
# survey_path = './data/BBDD_Dialogos.csv' #  chunksize=chunksize,
# survey = pd.read_csv(survey_path, low_memory=False)
# for survey in survey: break

survey_path = './data/BBDD_Dialogos.xlsx'
survey = pd.read_excel(survey_path, 'Hoja1')

filter_dialogue = pd.read_excel(survey_path, 'Filtro Diálogos')
filter_person = pd.read_excel(survey_path, 'Filtro Personas')

ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

filter_individual_path = './data/Filtro_individuales.xlsx'
filter_digitilized_individual = pd.read_excel(filter_individual_path, 'Hoja1')

## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [8]:
%%time
persons_table = create_table_persons(survey)

CPU times: user 3min 10s, sys: 640 ms, total: 3min 10s
Wall time: 3min 10s


In [9]:
persons_table.sample(2)

,id,diag_id,age,sex,level,comuna_id,age_range
4788,vl0skaoz8amn,enc_u_4560547262356713957,22,f,Técnica completa,11101,15-30
7198,rptzf5hsyxkq,enc_u_4547357304807853816,45,h,Básica completa,14201,45-60


### Dialogues

In [10]:
%%time
dialogues_table = create_table_dialogues(survey, filter_dialogue)

CPU times: user 2.67 s, sys: 20 ms, total: 2.69 s
Wall time: 2.71 s


In [11]:
dialogues_table.sample(2)

,id,date,init_time,end_time,location,address,comuna_id,n_members,group_name,valid
5789,enc_u_4597474986115689445,2020-03-06 00:00:00,09:00:00,11:30:00,comedor abierto del adulto mayor,"freire no 112, salamanca",4204,10,usuarios de oficina movil de bienes nacionales,True
1478,enc_u_4553573929489012701,2020-01-14 00:00:00,10:15:00,12:15:00,provincia de santiago,camino a melipilla 9229,13119,9,usuarios de oficina movil de bienes nacionales,True


### Persons Dialogues

In [12]:
%%time
persons_table, persons_dialogues_table = create_table_persons_dialogues(persons_table)

CPU times: user 67.7 ms, sys: 4 ms, total: 71.6 ms
Wall time: 70.3 ms


In [13]:
persons_dialogues_table.sample(2)

,person_id,diag_id
943,ny3+owl1yerx,enc_u_4560312205912440818
11624,zynostuf4n7c,enc_u_4520712042915289118


### Individuals

In [30]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey, filter_digitilized_individual)

CPU times: user 12.2 s, sys: 52.1 ms, total: 12.3 s
Wall time: 12.2 s


In [38]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,is_valid,age_range,online
5826,3245568,29-01-2020,25,4301,Media incompleta (incluyendo Media Técnica),True,15-30,False
1335,3041921,17-02-2020,45,11201,Técnica completa,True,45-60,False


### Emotions

In [76]:
%%time
emotions_table = create_table_emotions(survey, ind_survey_path, ind_online_survey)

CPU times: user 2min 56s, sys: 15.9 s, total: 3min 12s
Wall time: 3min 13s


In [137]:
emotions_table[(emotions_table['diag_id'] == '') & (emotions_table['ind_id'] == '')]

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online


### Emotion Pair

In [138]:
%%time
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

CPU times: user 14.6 s, sys: 55.6 ms, total: 14.6 s
Wall time: 14.6 s


In [139]:
emotion_pair.sample(3)

,id,emotion_id,word_1,word_2
120316,120317,21688,principalmente,ocurre
399270,399271,79521,destrozado,matado
281703,281704,51469,tema,protesta


### Country Needs

In [99]:
%%time
country_needs = create_table_country_needs(survey, ind_survey, ind_survey_path, ind_online_survey_path)

CPU times: user 6min, sys: 29.9 s, total: 6min 30s
Wall time: 6min 31s


In [140]:
country_needs.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
20261,19237,enc_u_4578407914127404624,,salud\neducacion\npensiones\ndelincuencia\neli...,"[salud, educacion, pensiones, delincuencia, el...",,,,"empresas: capacitar, mejorar la relacion de tr...","[empresas, capacitar, mejorar, relacion, traba...",empresas; medios de comunicacion,,False
2940,97577,,4092548,pension,[pension],,,,aumentar las pensiones de adulto mayor,"[aumentar, pensiones, adulto, mayor]",estado,,False


### Country Need Role Pair

In [141]:
%%time
country_need_role_pair = create_pair_token(country_needs, 'role_tokens', 'country_need_id')

CPU times: user 18.6 s, sys: 47.7 ms, total: 18.7 s
Wall time: 18.7 s


In [142]:
country_need_role_pair.sample(2)

,id,country_need_id,word_1,word_2
343514,343515,137597,proveer,recursos
79338,79339,25985,frente,probidad


### Country Need Explanation Pair

In [143]:
%%time
country_need_exp_pair = create_pair_token(country_needs, 'exp_tokens', 'country_need_id')

CPU times: user 19.8 s, sys: 48.1 ms, total: 19.8 s
Wall time: 19.8 s


In [144]:
country_need_exp_pair.sample(2)

,id,country_need_id,word_1,word_2
58039,58040,8682,sistema,gatillado
173843,173844,39936,puntos,reciclaje


### Family/Personal Needs

In [17]:
%%time
personal_needs = create_table_personal_needs(survey, ind_survey, ind_online_survey)

CPU times: user 3min 9s, sys: 19.4 s, total: 3min 28s
Wall time: 3min 29s


In [145]:
personal_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
538,35561,ENC_U_4562943770919287297,,entorno libre de contaminacion,"[entorno, libre, contaminacion]",-cuidado del entorno,"[cuidado, entorno]",entorno libre de contaminacion,0,False
4425,38580,ENC_U_4556212841195530421,,vivienda sociales,"[vivienda, sociales]",-que luego de tener una casa propia de no ser ...,"[luego, tener, casa, propia, ser, cobro, exces...",vivienda sociales,2,False
13030,64489,,3718630,la falta de oportunidades tanto laboral como e...,"[falta, oportunidades, laboral, salud]","el tener gente de mi familia sin trabajo, el n...","[tener, gente, familia, trabajo, tener, seguri...",la falta de oportunidades tanto laboral como e...,1,False


### Family/Personal Need Pair

In [146]:
%%time
personal_need_pair = create_pair_token(personal_needs, 'exp_tokens', 'personal_need_id')

CPU times: user 16.4 s, sys: 68 ms, total: 16.4 s
Wall time: 16.5 s


In [147]:
personal_need_pair.sample(3)

,id,personal_need_id,word_1,word_2
205513,205514,33204,participantes,estan
215396,215397,34701,horas,espera
432229,432230,79007,obtuve,ningun


### Contributions

In [85]:
%%time
contributions = create_table_contributions(survey, ind_survey, ind_online_survey)

CPU times: user 1min 19s, sys: 7.79 s, total: 1min 27s
Wall time: 1min 28s


In [90]:
contributions.sample(3)

,id,diag_id,ind_id,text,tokens,macro,is_online
0,0,enc_u_4602780640112847056,,asistir a dialogos,"[asistir, dialogos]",asistir a dialogos,False
1,1,enc_u_4602778880117363308,,conversando lo importante que es votar,"[conversando, importante, votar]",conversando lo importante que es votar,False
2,2,enc_u_4602777200118509403,,ayudar a entregar mejores valores a la juventud.,"[ayudar, entregar, mejores, valores, juventud]",ayudar a entregar mejores valores a la juventud.,False
4,4,enc_u_4602651983012551467,,luchar por igualdad de derecho de las mujeres,"[luchar, igualdad, derecho, mujeres]",luchar por igualdad de derecho de las mujeres,False
5,5,enc_u_4602650820112130336,,informarme por redes creibles,"[informarme, redes, creibles]",informarme por redes creibles,False
...,...,...,...,...,...,...,...
1799,82008,,0509e2ef02a033fc0efab1214e68be09,la mayor exigencia la desarrollo en mi persona...,"[mayor, exigencia, desarrollo, persona, respet...",la mayor exigencia la desarrollo en mi persona...,True
1800,82009,,c4960481a3875eb1cfbe32c33dc362c0,trabajar,[trabajar],trabajar,True
1801,82010,,2eac9b65b2b1ec5d134d6b45c83315fe,disponible para defender lo bueno que tiene ch...,"[disponible, defender, bueno, chile, disponibl...",disponible para defender lo bueno que tiene ch...,True
1802,82011,,ddc84f9d6d278cf3d32a0df820e01688,agente de paz,"[agente, paz]",agente de paz,True


## Saving `.csv`

In [36]:
out_dir = './out'
os.makedirs(out_dir, exist_ok=True)

In [60]:
contributions.to_csv(os.path.join(out_dir, 'contributions.csv'), index=False)
personal_need_pair.to_csv(os.path.join(out_dir, 'personal_need_pair.csv'), index=False)
personal_needs.to_csv(os.path.join(out_dir, 'personal_needs.csv'), index=False)
country_need_exp_pair.to_csv(os.path.join(out_dir, 'country_need_exp_pair.csv'), index=False)
country_need_role_pair.to_csv(os.path.join(out_dir, 'country_need_role_pair.csv'), index=False)
country_needs.to_csv(os.path.join(out_dir, 'country_needs.csv'), index=False)
emotion_pair.to_csv(os.path.join(out_dir, 'emotion_pair.csv'), index=False)
emotions_table.to_csv(os.path.join(out_dir, 'emotions.csv'), index=False)
individuals_table.to_csv(os.path.join(out_dir, 'individuals.csv'), index=False)
dialogues_table.to_csv(os.path.join(out_dir, 'dialogues.csv'), index=False)
persons_table.to_csv(os.path.join(out_dir, 'persons.csv'), index=False)
persons_dialogues_table.to_csv(os.path.join(out_dir, 'persons_dialogues.csv'), index=False)

## Saving `.sql`

In [61]:
con_to_sql(contributions,os.path.join(out_dir,'contributions.sql' ))
emo_to_sql(emotions_table,os.path.join(out_dir,'emotions.sql' ))
count_need_to_sql(country_needs,os.path.join(out_dir,'country_needs.sql' ))
per_need_to_sql(personal_needs,os.path.join(out_dir,'personal_need.sql' ))

In [80]:
emo_to_sql(emotions_table,os.path.join(out_dir,'emotions.sql' ))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# BORRAR LO DE ABAJO

In [5]:
emo_path = './data/emotions.csv' #  chunksize=chunksize,
emo_frame = pd.read_csv(emo_path, low_memory=False)

pers_path = './data/personal_needs.csv' #  chunksize=chunksize,
personal_need_frame = pd.read_csv(pers_path, low_memory=False)

con_path = './data/contributions.csv' #  chunksize=chunksize,
con_frame = pd.read_csv(con_path, low_memory=False)

coun_needs_path = './data/country_needs.csv' #  chunksize=chunksize,
coun_needs_frame = pd.read_csv(coun_needs_path, low_memory=False)

In [56]:
emo_path = './data/emotions.csv' #  chunksize=chunksize,
emo_frame = pd.read_csv(emo_path, low_memory=False)

In [135]:
personal_need_frame[(personal_need_frame['diag_id'] == '') & (personal_need_frame['ind_id'] == '')]

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online


In [148]:
personal_need_pair.to_csv(os.path.join(out_dir, 'personal_need_pair.csv'), index=False)
country_need_exp_pair.to_csv(os.path.join(out_dir, 'country_need_exp_pair.csv'), index=False)
country_need_role_pair.to_csv(os.path.join(out_dir, 'country_need_role_pair.csv'), index=False)
emotion_pair.to_csv(os.path.join(out_dir, 'emotion_pair.csv'), index=False)

In [57]:
emo_frame['name_tokens'] = emo_frame['name_tokens'].apply(lambda x: tt.clean_alt_list(x))
emo_frame['exp_tokens'] = emo_frame['exp_tokens'].apply(lambda x: tt.clean_alt_list(x))
emo_frame = emo_frame.replace({'nr':'','nan':'', 'NR':'', 'NaN':'', np.nan:''})
emo_frame['diag_id'] = tt.to_unicode(emo_frame['diag_id'])

In [10]:
personal_need_frame['name_tokens'] = personal_need_frame['name_tokens'].apply(lambda x: tt.clean_alt_list(x))
personal_need_frame['exp_tokens'] = personal_need_frame['exp_tokens'].apply(lambda x: tt.clean_alt_list(x))
personal_need_frame['priority'] = personal_need_frame['priority'].astype(int)
personal_need_frame = personal_need_frame.replace({'nr':'','nan':'', 'NR':'', 'NaN':'', np.nan:''})

In [82]:
con_frame['tokens'] = con_frame['tokens'].apply(lambda x: tt.clean_alt_list(x))
con_frame = con_frame.replace({'nr':'','nan':'', 'NR':'', 'NaN':'', np.nan:''})

In [26]:
coun_needs_frame = coun_needs_frame.replace({'nr':'','nan':'', 'NR':'', 'NaN':'', np.nan:''})
coun_needs_frame['priority'] = coun_needs_frame['priority'].apply(lambda x: tt.str_to_int(x))
coun_needs_frame['name_tokens'] = coun_needs_frame['name_tokens'].apply(lambda x: tt.clean_alt_list(x))
coun_needs_frame['exp_tokens'] = coun_needs_frame['exp_tokens'].apply(lambda x: tt.clean_alt_list(x))
coun_needs_frame['role_tokens'] = coun_needs_frame['role_tokens'].apply(lambda x: tt.clean_alt_list(x))


In [91]:
con_to_sql(contributions,'./data/contributions.sql' )

In [65]:
emo_to_sql(emo_frame,'./data/emotions.sql')

In [124]:
count_need_to_sql(country_needs,'./data/country_needs.sql')

In [136]:
per_need_to_sql(personal_need_frame,'./data/personal_need.sql')